In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
# from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## 1. 문서 로드

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader, PDFPlumberLoader, UnstructuredPDFLoader

In [4]:
file_path = 'C:\Potenup\Moms-Diary-Chatbot\data\Pregnancy Book 2025.pdf'

In [5]:
print("PyPDFLoader")
loader = PyPDFLoader(file_path)
docs = loader.load()

PyPDFLoader


In [7]:
print(docs[8].page_content)

Your pregnancy at a glance
7
• Find out about antenatal education (see Antenatal care on page 57).
• Start to think about how you want to feed your baby (see Feeding your baby on page 121).
• Make sure you are wearing a supportive bra. Your breasts will probably increase in size 
during pregnancy so you need to make sure you are wearing the right sized bra.
• If you have been feeling sick and tired, you will probably start to feel better around this time.
• Your pregnancy may just be beginning to show. This varies a lot from woman  
to woman.
12–16 weeks
• You may start to feel your baby move (see How your baby develops on 
page 23).
• Your tummy will begin to get bigger and you will need looser clothes.
• You may feel a surge of energy.
• Try to do your pregnancy exercises regularly (see Keeping active on 
page 45).
• Your midwife or doctor should:
 -  review, discuss and record the results of any screening tests;
 -  measure your blood pressure and test your urine for protein; 
 -  c

In [29]:
print(docs[13].page_content)

12
Hormones
Both men and women 
have hormones, which are 
chemicals that circulate in 
the bloodstream. They carry 
messages to different parts of 
the body and result in certain 
changes taking place. Female 
hormones, which include 
oestrogen and progesterone, 
control many of the events 
of a woman’s monthly cycle, 
such as the release of eggs 
from her ovaries and the 
thickening of her uterus lining.
During pregnancy, your 
hormone levels change. As 
soon as you have conceived, 
the amount of oestrogen and 
progesterone in your blood 
increases. This causes the 
uterus lining to build up, the 
blood supply to your uterus 
and breasts to increase and 
the muscles of your uterus 
to relax to make room for the 
growing baby.
The increase in hormone levels 
can affect how you feel. You 
may have mood swings, feel 
tearful or be easily irritated. For 
a while you may feel that you 
cannot control your emotions, 
but these symptoms should 
ease after the first three 
months of your preg

## 2. 청크 나누기

In [8]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunks = splitter.split_documents(docs)
print(len(chunks))

668


In [9]:
print(chunks[20].page_content)

with	spina	bifida,	or	if	you	
have coeliac disease, sickle 
cell disease, diabetes, 
are living with overweight 
or obesity or take anti-
epileptic medicines, ask 
your GP or midwife for 
more advice. You will need 
to take a bigger dose of 
folic acid that requires a 
prescription.
• If you have a health 
condition, for example 
mental health problems, 
diabetes or a family 
history of any inherited 
diseases, talk to your GP 
or a specialist before you 
try to get pregnant.
• Talk to your GP or a 
healthcare professional if 
you have any concerns or 
need support.
PHDG youtube - 
Planning your pregnancy
www.youtube.com/ 
watch?v=R2VmFqMm6Z8 
&t=6s


## 3. 벡터 스토어 

In [10]:
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
db_path = "../vectorstore/chroma_pregnancy_2025"
collection_name = "pregnancy_2025"

In [11]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate

vectorstore = Chroma.from_documents(
    documents = chunks,
    embedding = embedding,
    persist_directory = db_path,
    collection_name = collection_name
)

## 4. 검색기

### 벡터 스토어 불러오기

In [ ]:
# 벡터저장소가 이미 있는 상황
vectorstore = Chroma(
    persist_directory=db_path,
    collection_name=collection_name,
    embedding_function=embedding
)

In [ ]:
# retriever 구성하기
retriever = vectorstore.as_retriever(
    search_kwargs = {"k" : 5}
)

In [ ]:
vectorstore._collection.count()

668

## 5. LLM Chain 구축

In [53]:
# 5. 프롬프트 구성하기
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
    너는 산부인과 경력 30년 이상의 전문의야
    산모의 궁금한 점을 주어진 컨텍스트를 근거로 정확하게 대답해

     [작성 규칙]
     - 컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라
     - 한글과 영어로 대답하라
     
     [출처 작성 규칙]
     - 출처 내용을 있는 그대로 작성할 것
     - 있는 그대로 작성 할 것
     - 대답에는 어떤 문서의 몇 페이지를 근거하고 있는지 각 문장별로 괄호 안에 작성해라
     - 여러 문서를 근거하고 있을 경우 누락 없이 모두 작성하라

     [출력 형식]
     전문의로서의 답변
     출처 내용 리스트 : 내용(출처)

     [컨텍스트]
     {context}
"""
rag_prompt = ChatPromptTemplate.from_messages([
    ("system" , system_prompt),
    ("human", "{question}")
])

In [ ]:
# 2. 모델 선택
model = ChatOpenAI(
    temperature=0,
    model = "gpt-4.1-mini",
    verbose=True
)

# 3. output parser 선택
outputparser = StrOutputParser()

In [ ]:
# 문서를 합치는 함수
def format_docs(docs):
    return "\n\---\n\n".join([f"컨텐츠: " + doc.page_content + f"\n페이지: {doc.metadata.get('page_label')}" for doc in docs])

In [54]:
# 4. chain 생성
chain = rag_prompt | model | outputparser

rag_chain = (
    {
        "context" : retriever | RunnableLambda(format_docs),
        "question" : RunnablePassthrough()
    }
    | chain
)

In [40]:
result = rag_chain.invoke("내 아이가 여자인지 남자인지 언제 알 수 있어?")
print(result)

전문의로서의 답변:

아기의 성별은 수정 시 아버지의 정자가 X염색체를 가지고 있으면 여자(XX), Y염색체를 가지고 있으면 남자(XY)로 결정됩니다(페이지 14). 임신 중 아기의 성별을 확인하는 방법 중 하나는 양수검사(amniocentesis)입니다. 이 검사는 양수에서 아기와 동일한 염색체를 가진 세포를 채취하여 검사하는 것으로, 보통 다운증후군 등 염색체 이상을 검사하는 데 사용되며, 검사 결과는 보통 3일에서 최대 3주까지 걸릴 수 있습니다. 이 검사에서 아기의 성별도 알 수 있으므로, 검사 시기에 미리 성별을 알고 싶은지 의료진에게 알려야 합니다(페이지 71).

따라서, 일반적으로 임신 초기에는 성별을 알기 어렵고, 양수검사나 초음파 검사 등을 통해 임신 중기 이후에 성별을 확인할 수 있습니다. 다만, 양수검사는 유산 위험이 0.5~1% 있으므로 신중히 결정해야 합니다(페이지 71).

출처 내용 리스트  
1. 아기의 성별 결정과 염색체 (페이지 14)  
2. 양수검사와 성별 확인 가능 시기 및 위험성 (페이지 71)


In [41]:
result = rag_chain.invoke("임신 중기 이후 성별을 확인할 수 있다는데 그 시기가 언제쯤이야?")
print(result)

임신 중기 이후에 아기의 성별을 확인할 수 있는 시기는 일반적으로 양수검사(amniocentesis)나 초음파 검사를 통해 가능합니다. 양수검사는 보통 임신 15~20주 사이에 시행하며, 이 검사에서 아기의 염색체를 분석하여 성별을 알 수 있습니다. 검사 결과는 보통 3일에서 3주 정도 걸리며, 이때 아기의 성별을 알려줄 수 있습니다(페이지 71).

또한, 초음파 검사는 보통 임신 18~22주 사이에 실시하는 중기 초음파 검사에서 아기의 성별을 확인할 수 있습니다. 다만, 초음파로 성별을 확인하는 것은 검사 시기의 아기의 자세나 해상도에 따라 정확도가 달라질 수 있습니다.

따라서 임신 중기 이후, 대략 15주에서 22주 사이에 아기의 성별을 확인할 수 있다고 보시면 됩니다.

출처 내용 리스트  
1. 양수검사 및 성별 확인 (페이지 71)


In [42]:
result = rag_chain.invoke("임신 초기에 하면 좋은 운동을 알려줘")
print(result)

임신 초기(1~12주)에는 가벼운 운동부터 시작하는 것이 좋습니다. 걷기는 임신 초기에 시작하기 좋은 운동이며, 운동 강도는 너무 무리하지 않고 대화가 가능한 정도로 유지하는 것이 안전합니다. 또한, 너무 더운 환경에서 운동하는 것은 피해야 합니다. 복부에 직접적인 압박이 가해지는 접촉 스포츠나 복근 운동(윗몸 일으키기 등)은 피하는 것이 좋습니다. 수영, 걷기, 춤 등이 임신 초기 운동으로 적합하며, 운동 전후로 충분한 준비운동과 정리운동을 하는 것이 중요합니다. 하루 30분, 주 4회 정도의 활동을 목표로 하되, 몸 상태에 따라 조절하며 무리가 가지 않도록 해야 합니다(47~48페이지).

요약하면, 임신 초기에 추천되는 운동은 다음과 같습니다:
- 걷기
- 수영
- 가벼운 춤
- 자전거 타기(실내 자전거 포함)
- 로잉 머신 사용(무리가 가지 않는 범위 내에서)

운동 중 불편함이 느껴지면 즉시 중단하고 전문가와 상담하는 것이 좋습니다(47페이지).

전문의로서 권고드립니다.

출처 내용 리스트  
1. 임신 중 운동 가이드 및 주의사항 (47~48페이지)  
2. 임신 초기 운동 권장 및 주의사항 (47페이지)


In [51]:
result = rag_chain.invoke("다리가 아파서 힘든데 어떻게 하면 좋을까?")
print(result)

근거 없음

출처 내용 리스트 : 다리가 아파서 힘든 경우에 대한 내용 없음(컨텍스트 전반)


In [55]:
result = rag_chain.invoke("배가 당기는 느낌이 드는데 이유가 뭘까?")
print(result)

전문의로서의 답변  
배가 당기는 느낌은 임신 중에 흔히 경험할 수 있는 증상입니다. 이는 자궁이 커지면서 주변 조직과 인대가 늘어나거나 당겨지는 느낌일 수 있습니다. 또한, 호르몬 변화와 피로감도 이러한 불편감을 유발할 수 있습니다. 특히 임신 초기 3개월 동안은 호르몬 변화로 인해 피로하고 감정적으로 불안정해질 수 있으며, 이로 인해 신체적인 불편감도 더 크게 느껴질 수 있습니다. 만약 통증이 심하거나 출혈, 발열 등의 다른 증상이 동반된다면 즉시 의료진과 상담하는 것이 중요합니다.  

Abdominal pulling sensation is a common symptom during pregnancy. It can be caused by the stretching or pulling of the uterus and surrounding ligaments as the uterus grows. Hormonal changes and fatigue can also contribute to this discomfort. Especially in the first three months, hormonal fluctuations can make you feel tired and emotionally sensitive, which may increase physical discomfort. However, if the pain is severe or accompanied by bleeding, fever, or other symptoms, it is important to consult a healthcare professional immediately.  

출처 내용 리스트 :  
- "Hormonal changes taking place in your body can make you feel tired, nauseous, emotional and upset – particularly in the first three months." (페이지 93)  
- "You

In [56]:
result = rag_chain.invoke("우울 증상에 대해서 알려줘")
print(result)

전문의로서의 답변  
컨텍스트에 우울 증상에 관한 내용이 없습니다. 따라서 우울 증상에 대해 답변할 수 없습니다.  
There is no information about depressive symptoms in the provided context. Therefore, I cannot provide an answer regarding depressive symptoms.

출처 내용 리스트 : 근거 없음


In [57]:
result = rag_chain.invoke("산후 우울에 대해서 알려줘")
print(result)

근거 없음  
No evidence in the provided context regarding postpartum depression.  

출처 내용 리스트 : 없음


In [59]:
result = rag_chain.invoke("임신 중 불안에 대해서 알려줘")
print(result)

전문의로서의 답변  
임신 중 불안은 매우 흔한 감정이며, 임신과 출산에 대한 걱정이나 미래에 대한 불확실성 때문에 발생할 수 있습니다. 임신 기간 동안 기분 변화가 있을 수 있으며, 이러한 감정을 혼자서 견디지 말고 주변 사람이나 의료진에게 알리는 것이 중요합니다. 특히 이전에 정신 건강 문제를 겪은 적이 있다면 임신 중 우울증이나 불안 증상이 나타날 위험이 높으므로, 임신 초기에 조산사나 의사에게 과거 병력을 알리는 것이 필요합니다. 또한, 출산에 대한 걱정을 줄이기 위해 산전 교육을 받거나 출산과 관련된 정보를 미리 알아보는 것이 도움이 됩니다. 파트너나 가까운 사람이 함께 출산 준비를 하면서 지지해주는 것도 불안을 완화하는 데 중요합니다.  
Anxiety during pregnancy is a common experience, often related to worries about childbirth and uncertainty about the future. Mood changes can occur during pregnancy, and it is important not to suffer alone but to inform someone close or healthcare professionals about how you feel. If you have a history of mental health problems, you may be at higher risk of developing depressive or anxiety symptoms during pregnancy, so it is important to inform your midwife or doctor early in pregnancy. Attending antenatal education and learning about labor and birth can help reduce anxiety. Support from your partner or someone close during 

- 영어 데이터에 대해서 한글로 질문했을 때 더 답변 잘 하게 하려면?
